In [ ]:
! pip install python-dotenv
! pip install pydub
! pip install pandas
! pip install psycopg2
! pip install pyannote.audio
! pip install google-auth google-auth-httplib2 google-api-python-client google-auth-oauthlib
! pip install gdown
! pip install python-docx

In [ ]:
import sys
sys.path.append('../util')

from util import read_spreadsheet
from util import get_max_db_id

In [ ]:
import os
import gdown
from docx import Document

def docx_to_txt(docx_path):
        doc = Document(docx_path)
        fullText = []
        for para in doc.paragraphs:
            fullText.append(para.text)
        return '\n'.join(fullText)

def download_etext(gd_url,file_name):
    if not os.path.exists('etexts'):
        os.makedirs('etexts')

    if not os.path.exists('docx'):
        os.makedirs('docx')
    
    if os.path.exists(f'etexts/{file_name}.txt'):
        return
    docx_url, _ = os.path.split(gd_url)
    docx_url = os.path.join(docx_url, 'export?format=docx')
    docx_path = gdown.download(docx_url, output=f'docx/{file_name}.docx', quiet=False, fuzzy=True)
    # Convert the .docx file to text
    text = docx_to_txt(docx_path)
    # Create a .txt path with the same name as the .docx file
    txt_path = os.path.join('etexts/', file_name + '.txt')
    # Save the text to a .txt file
    with open(txt_path, 'w',encoding='utf-8') as f:
        f.write(text.replace('\n', ' '))

In [ ]:
# Configuration
ab_sheet_id = "1yKSzConuVWo8BuMDs2mabF5iiBKUz2wF--LIabFN6QE"
sr_no_start = 645
sr_no_end = 691

# Column names
col_file_name = 'ID'
col_gd_url = 'Etext link'
col_sr_no = 'z'

# Read the spreadsheet data
df = read_spreadsheet(sheet_id=ab_sheet_id)

# Iterate over the DataFrame rows
for index, row in df.iterrows():
    file_name = row[col_file_name]
    gd_url = row[col_gd_url]
    sr_no = row[col_sr_no]

    # Ensure the Sr_no is within the desired range before downloading
    if sr_no_start <= sr_no <= sr_no_end:
        print(f"Downloading: {file_name} from {gd_url}, Sr_no: {sr_no}")
        download_etext(gd_url=gd_url, file_name=file_name)
